<a href="https://colab.research.google.com/github/Qyeler/AI_Learning_Week_2025_Nikita_Nazarov/blob/main/NikitaNazarov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import os
import requests
import json
import pandas as pd
import time

OPENROUTER_API_KEY = ""

if OPENROUTER_API_KEY is None:
    OPENROUTER_API_KEY = "YOUR_OPENROUTER_KEY_HERE".strip()

BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

def simple_test_call():
    """
    Минимальный тест: запрос к openai/gpt-4.1-nano через OpenRouter.
    Никаких русских заголовков, только ASCII.
    """
    headers = {
        "Authorization": "Bearer " + OPENROUTER_API_KEY,
        "Content-Type": "application/json",
    }

    payload = {
        "model": "openai/gpt-4.1-nano",
        "messages": [
            {"role": "user", "content": "Скажи привет одним коротким предложением."}
        ],
    }

    resp = requests.post(
        BASE_URL,
        headers=headers,
        json=payload,
        timeout=60,
    )
    print("Status:", resp.status_code)
    print("Raw text:", resp.text[:500])
    return resp

resp = simple_test_call()

Status: 200
Raw text: 
         

         

         

         

         
{"id":"gen-1763936311-TuIHhekd4qQX2NYfD2pT","provider":"OpenAI","model":"openai/gpt-4.1-nano","object":"chat.completion","created":1763936311,"choices":[{"logprobs":null,"finish_reason":"stop","native_finish_reason":"completed","index":0,"message":{"role":"assistant","content":"Привет!","refusal":null,"reasoning":null}}],"usage":{"prompt_tokens":17,"completion_tokens":4,"total_tokens":21,"completion_tokens_details":{"reasoning_tokens":0}}}


Нашел отзывы:

In [ ]:
reviews = [
    "Купил DJI Mini 3 Pro. Камера отличная, но батарея садится быстро. За свои деньги норм.",
    "Дрон летает стабильно даже при ветре, но приложение иногда зависает, приходится перезапускать.",
    "Разочарован. Через неделю полётов сломался двигатель, сервис тянет с ремонтом.",
    "Очень лёгкий и компактный дрон, удобно брать в поездки. Качество видео 4K радует.",
    "Шумный, но мощный. Поднимается высоко, сигнал не пропадает. Жаль, что нет датчиков обхода препятствий.",
    "Игрушка для ребенка. Камера так себе, но управлять легко, ребёнок доволен.",
    "Запас хода по батарее маленький — максимум 18 минут. Для съёмки мероприятий этого мало.",
    "Качество сборки отличное, ничего не люфтит. Пульт удобный, всё интуитивно.",
    "После обновления прошивки дрон начал терять связь на средней дистанции, стало страшно далеко летать.",
    "Отличный вариант для первого дрона: простое управление, хороший авто-режим взлёта и посадки.",
    "Гимбал стабилизирует отлично, картинка ровная даже при резких манёврах.",
    "Цена завышена, особенно если смотреть на конкурентов. Если бы стоил дешевле — поставил бы 5."
]

MODELS = [
    {"id": "openai/gpt-4.1-nano",  "name": "GPT-4.1 Nano"},
    {"id": "mistralai/mistral-small-24b-instruct-2501:free", "name": "Mistral Small 3 (free)"},
    {"id": "meta-llama/llama-3.3-70b-instruct:free", "name": "Llama 3.3 70B (free)"},
]

analysis_instruction = (
    "Ты — AI-анализатор отзывов о дронах и квадрокоптерах.\n\n"
    "Для данного отзыва тебе нужно:\n"
    "1) Определи тональность: \"положительный\", \"нейтральный\" или \"отрицательный\".\n"
    "2) Определи основное преимущество или проблему (main_topic) — короткая фраза.\n"
    "3) Если есть проблема, опиши её (issue) одним коротким предложением, иначе оставь пустую строку.\n"
    "4) Определи общую оценку отзыва от 1 до 5 (целое число).\n\n"
    "Верни ТОЛЬКО JSON такого вида:\n"
    "{\n"
    "  \"sentiment\": \"положительный/нейтральный/отрицательный\",\n"
    "  \"main_topic\": \"строка\",\n"
    "  \"issue\": \"строка\",\n"
    "  \"rating\": 1\n"
    "}\n"
    "Без комментариев и пояснений вокруг."
)

In [ ]:
def extract_json(text: str) -> dict:
    """
    Вытаскивает JSON из текста модели (между первой '{' и последней '}').
    Если модель написала что-то до/после — отрежем.
    """
    start = text.find("{")
    end = text.rfind("}")
    if start == -1 or end == -1:
        raise ValueError("Не найден JSON в ответе модели:\n" + text)
    json_str = text[start:end+1]
    return json.loads(json_str)


def call_openrouter(model_id: str, system_prompt: str, user_text: str, max_retries: int = 3) -> str:
    """
    Запрос к OpenRouter с простым retry на 429.
    """
    headers = {
        "Authorization": "Bearer " + OPENROUTER_API_KEY,
        "Content-Type": "application/json",
    }

    payload = {
        "model": model_id,
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user",   "content": user_text},
        ],
    }

    for attempt in range(1, max_retries + 1):
        resp = requests.post(
            BASE_URL,
            headers=headers,
            json=payload,
            timeout=60,
        )

        if resp.status_code == 429:
            wait_time = 5 * attempt
            print(f"  ⚠️ 429 Too Many Requests для {model_id}, попытка {attempt}/{max_retries}, ждём {wait_time} сек...")
            time.sleep(wait_time)
            continue

        resp.raise_for_status()
        data = resp.json()
        return data["choices"][0]["message"]["content"]

    raise RuntimeError(f"Не удалось получить ответ от {model_id} после {max_retries} попыток (429).")


def analyze_review_with_model(model_id: str, review_text: str) -> dict:
    """
    Анализирует один отзыв: вызывает модель и парсит JSON.
    """
    raw_text = call_openrouter(model_id, analysis_instruction, review_text).strip()

    if raw_text.startswith("```"):
        raw_text = raw_text.strip("`")
        if raw_text.lower().startswith("json"):
            raw_text = raw_text[4:].strip()

    return extract_json(raw_text)

In [ ]:
MODEL_ID = "openai/gpt-4.1-nano"
MODEL_NAME = "GPT-4.1 Nano"

rows_nano = []

print(f"Обрабатываем модель: {MODEL_NAME} ({MODEL_ID})")

for i, review in enumerate(reviews):
    print(f"  Отзыв {i+1}/{len(reviews)}")
    analysis = analyze_review_with_model(MODEL_ID, review)


    rows_nano.append({
        "review_id": i,
        "review_text": review,
        "model_id": MODEL_ID,
        "model_name": MODEL_NAME,
        "sentiment": analysis.get("sentiment"),
        "main_topic": analysis.get("main_topic"),
        "issue": analysis.get("issue"),
        "rating": analysis.get("rating"),
    })

    time.sleep(1)

df_model1 = pd.DataFrame(rows_nano)
df_model1


Обрабатываем модель: GPT-4.1 Nano (openai/gpt-4.1-nano)
  Отзыв 1/12
  Отзыв 2/12
  Отзыв 3/12
  Отзыв 4/12
  Отзыв 5/12
  Отзыв 6/12
  Отзыв 7/12
  Отзыв 8/12
  Отзыв 9/12
  Отзыв 10/12
  Отзыв 11/12
  Отзыв 12/12


,review_id,review_text,model_id,model_name,sentiment,main_topic,issue,rating
0,0,"Купил DJI Mini 3 Pro. Камера отличная, но бата...",openai/gpt-4.1-nano,GPT-4.1 Nano,нейтральный,качество камеры,батарея садится быстро,3
1,1,"Дрон летает стабильно даже при ветре, но прило...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,стабильность полетов,приложение иногда зависает,4
2,2,Разочарован. Через неделю полётов сломался дви...,openai/gpt-4.1-nano,GPT-4.1 Nano,отрицательный,качество двигателя,сломался двигатель после недели использования,2
3,3,"Очень лёгкий и компактный дрон, удобно брать в...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,компактность и качество видео,,4
4,4,"Шумный, но мощный. Поднимается высоко, сигнал ...",openai/gpt-4.1-nano,GPT-4.1 Nano,нейтральный,мощность и высота полёта,отсутствие датчиков обхода препятствий,3
5,5,"Игрушка для ребенка. Камера так себе, но управ...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,управляемость,качество камеры так себе,4
6,6,Запас хода по батарее маленький — максимум 18 ...,openai/gpt-4.1-nano,GPT-4.1 Nano,отрицательный,время работы батареи,единичное время полета недостаточно для съемки...,2
7,7,"Качество сборки отличное, ничего не люфтит. Пу...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,качество сборки и управление,,5
8,8,После обновления прошивки дрон начал терять св...,openai/gpt-4.1-nano,GPT-4.1 Nano,отрицательный,Проблемы со связью после обновления прошивки,Дрон начал терять связь на средней дистанции,2
9,9,Отличный вариант для первого дрона: простое уп...,openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,простой старт и управление,,4


In [ ]:
MODEL_ID_2 = "x-ai/grok-4.1-fast"
MODEL_NAME_2 = "Grok 4.1 Fast"

rows_model2 = []

print(f"Обрабатываем модель: {MODEL_NAME_2} ({MODEL_ID_2})")

for i, review in enumerate(reviews):
    print(f"  Отзыв {i+1}/{len(reviews)}")
    analysis = analyze_review_with_model(MODEL_ID_2, review)

    rows_model2.append({
        "review_id": i,
        "review_text": review,
        "model_id": MODEL_ID_2,
        "model_name": MODEL_NAME_2,
        "sentiment": analysis.get("sentiment"),
        "main_topic": analysis.get("main_topic"),
        "issue": analysis.get("issue"),
        "rating": analysis.get("rating"),
    })

    time.sleep(3)

df_model2 = pd.DataFrame(rows_model2)
df_model2


Обрабатываем модель: Grok 4.1 Fast (x-ai/grok-4.1-fast)
  Отзыв 1/12
  Отзыв 2/12
  Отзыв 3/12
  Отзыв 4/12
  Отзыв 5/12
  Отзыв 6/12
  Отзыв 7/12
  Отзыв 8/12
  Отзыв 9/12
  Отзыв 10/12
  Отзыв 11/12
  Отзыв 12/12


,review_id,review_text,model_id,model_name,sentiment,main_topic,issue,rating
0,0,"Купил DJI Mini 3 Pro. Камера отличная, но бата...",x-ai/grok-4.1-fast,Grok 4.1 Fast,нейтральный,"Отличная камера, но быстрая разрядка батареи",Батарея садится быстро.,3
1,1,"Дрон летает стабильно даже при ветре, но прило...",x-ai/grok-4.1-fast,Grok 4.1 Fast,нейтральный,стабильный полет при ветре,"Приложение иногда зависает, приходится перезап...",4
2,2,Разочарован. Через неделю полётов сломался дви...,x-ai/grok-4.1-fast,Grok 4.1 Fast,отрицательный,"Сломался двигатель через неделю, задержки в се...","Двигатель сломался через неделю полетов, серви...",1
3,3,"Очень лёгкий и компактный дрон, удобно брать в...",x-ai/grok-4.1-fast,Grok 4.1 Fast,положительный,лёгкий и компактный дрон с качественным 4K видео,,5
4,4,"Шумный, но мощный. Поднимается высоко, сигнал ...",x-ai/grok-4.1-fast,Grok 4.1 Fast,нейтральный,"Мощный, поднимается высоко, но шумный",Отсутствуют датчики обхода препятствий.,4
5,5,"Игрушка для ребенка. Камера так себе, но управ...",x-ai/grok-4.1-fast,Grok 4.1 Fast,положительный,"Легкое управление, ребенок доволен",Камера посредственного качества,4
6,6,Запас хода по батарее маленький — максимум 18 ...,x-ai/grok-4.1-fast,Grok 4.1 Fast,отрицательный,маленький запас хода батареи,Максимум 18 минут полёта недостаточно для съём...,1
7,7,"Качество сборки отличное, ничего не люфтит. Пу...",x-ai/grok-4.1-fast,Grok 4.1 Fast,положительный,Отличное качество сборки и удобный пульт,,5
8,8,После обновления прошивки дрон начал терять св...,x-ai/grok-4.1-fast,Grok 4.1 Fast,отрицательный,потеря связи после обновления прошивки,Дрон начал терять связь на средней дистанции п...,2
9,9,Отличный вариант для первого дрона: простое уп...,x-ai/grok-4.1-fast,Grok 4.1 Fast,положительный,простое управление и хороший авто-режим взлёта...,,5


In [ ]:
MODEL_ID_3 = "deepseek/deepseek-chat"
MODEL_NAME_3 = "DeepSeek Chat"

rows_model3 = []

print(f"Обрабатываем модель: {MODEL_NAME_3} ({MODEL_ID_3})")

for i, review in enumerate(reviews):
    print(f"  Отзыв {i+1}/{len(reviews)}")
    analysis = analyze_review_with_model(MODEL_ID_3, review)

    rows_model3.append({
        "review_id": i,
        "review_text": review,
        "model_id": MODEL_ID_3,
        "model_name": MODEL_NAME_3,
        "sentiment": analysis.get("sentiment"),
        "main_topic": analysis.get("main_topic"),
        "issue": analysis.get("issue"),
        "rating": analysis.get("rating"),
    })

    time.sleep(2)

df_model3 = pd.DataFrame(rows_model3)
df_model3


Обрабатываем модель: DeepSeek Chat (deepseek/deepseek-chat)
  Отзыв 1/12
  Отзыв 2/12
  Отзыв 3/12
  Отзыв 4/12
  Отзыв 5/12
  Отзыв 6/12
  Отзыв 7/12
  Отзыв 8/12
  Отзыв 9/12
  Отзыв 10/12
  Отзыв 11/12
  Отзыв 12/12


,review_id,review_text,model_id,model_name,sentiment,main_topic,issue,rating
0,0,"Купил DJI Mini 3 Pro. Камера отличная, но бата...",deepseek/deepseek-chat,DeepSeek Chat,нейтральный,камера и батарея,батарея садится быстро,3
1,1,"Дрон летает стабильно даже при ветре, но прило...",deepseek/deepseek-chat,DeepSeek Chat,нейтральный,стабильность полета,приложение иногда зависает,4
2,2,Разочарован. Через неделю полётов сломался дви...,deepseek/deepseek-chat,DeepSeek Chat,отрицательный,поломка двигателя,"двигатель сломался через неделю, сервис медлит...",1
3,3,"Очень лёгкий и компактный дрон, удобно брать в...",deepseek/deepseek-chat,DeepSeek Chat,положительный,лёгкость и компактность,,5
4,4,"Шумный, но мощный. Поднимается высоко, сигнал ...",deepseek/deepseek-chat,DeepSeek Chat,нейтральный,мощность и сигнал,нет датчиков обхода препятствий,3
5,5,"Игрушка для ребенка. Камера так себе, но управ...",deepseek/deepseek-chat,DeepSeek Chat,положительный,легкость управления,Камера так себе,4
6,6,Запас хода по батарее маленький — максимум 18 ...,deepseek/deepseek-chat,DeepSeek Chat,отрицательный,малый запас хода батареи,18 минут работы недостаточно для съёмки меропр...,2
7,7,"Качество сборки отличное, ничего не люфтит. Пу...",deepseek/deepseek-chat,DeepSeek Chat,положительный,качество сборки и удобство пульта,,5
8,8,После обновления прошивки дрон начал терять св...,deepseek/deepseek-chat,DeepSeek Chat,отрицательный,потеря связи,Дрон теряет связь на средней дистанции после о...,2
9,9,Отличный вариант для первого дрона: простое уп...,deepseek/deepseek-chat,DeepSeek Chat,положительный,простое управление и авто-режимы,,5


In [ ]:
frames = [df_nano]

if 'df_model2' in globals() and not df_model2.empty:
    frames.append(df_model2)

if 'df_model3' in globals() and not df_model3.empty:
    frames.append(df_model3)

df_all = pd.concat(frames, ignore_index=True)
df_all


,review_id,review_text,model_id,model_name,sentiment,main_topic,issue,rating
0,0,"Купил DJI Mini 3 Pro. Камера отличная, но бата...",openai/gpt-4.1-nano,GPT-4.1 Nano,нейтральный,качество камеры,батарея садится быстро,3
1,1,"Дрон летает стабильно даже при ветре, но прило...",openai/gpt-4.1-nano,GPT-4.1 Nano,нейтральный,стабильность полета,приложение зависает и требует перезапуска,3
2,2,Разочарован. Через неделю полётов сломался дви...,openai/gpt-4.1-nano,GPT-4.1 Nano,отрицательный,качество двигателя,"сломался двигатель через неделю эксплуатации, ...",2
3,3,"Очень лёгкий и компактный дрон, удобно брать в...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,компактность и качество видео,,4
4,4,"Шумный, но мощный. Поднимается высоко, сигнал ...",openai/gpt-4.1-nano,GPT-4.1 Nano,нейтральный,мощность и высота полёта,нет датчиков обхода препятствий,3
5,5,"Игрушка для ребенка. Камера так себе, но управ...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,удобство управления,камера так себе,4
6,6,Запас хода по батарее маленький — максимум 18 ...,openai/gpt-4.1-nano,GPT-4.1 Nano,отрицательный,время работы батареи,Запас хода по батарее слишком короткий,2
7,7,"Качество сборки отличное, ничего не люфтит. Пу...",openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,качество сборки и управление,,5
8,8,После обновления прошивки дрон начал терять св...,openai/gpt-4.1-nano,GPT-4.1 Nano,отрицательный,проблемы с связью после обновления,Дрон теряет связь на средней дистанции,2
9,9,Отличный вариант для первого дрона: простое уп...,openai/gpt-4.1-nano,GPT-4.1 Nano,положительный,легкое управление,,4


In [ ]:
model_rating_summary = df_all.groupby("model_name")["rating"].mean().reset_index()
print(model_rating_summary)

sentiment_summary = df_all.pivot_table(
    index="model_name",
    columns="sentiment",
    values="review_id",
    aggfunc="count",
    fill_value=0
).reset_index()

print(sentiment_summary)


      model_name    rating
0  DeepSeek Chat  3.416667
1   GPT-4.1 Nano  3.250000
2  Grok 4.1 Fast  3.500000
sentiment     model_name  нейтральный  отрицательный  положительный
0          DeepSeek Chat            3              4              5
1           GPT-4.1 Nano            3              4              5
2          Grok 4.1 Fast            3              4              5


## 3. Таблица сравнения моделей

### 3.1. Средний рейтинг по моделям

| Модель         | Средний рейтинг |
|----------------|-----------------|
| GPT-4.1 Nano   | 3.25            |
| Grok 4.1 Fast  | 3.50            |
| DeepSeek Chat  | 3.42            |

Разброс по среднему рейтингу небольшой (от 3.25 до 3.50), все модели оценивают отзывы похоже по строгости.

### 3.2. Распределение тональностей

| Модель         | Нейтральный | Отрицательный | Положительный |
|----------------|------------:|--------------:|--------------:|
| GPT-4.1 Nano   | 3           | 4             | 5             |
| Grok 4.1 Fast  | 3           | 4             | 5             |
| DeepSeek Chat  | 3           | 4             | 5             |

Все три модели показывают одинаковое распределение тональностей: 5 положительных, 4 отрицательных и 3 нейтральных отзыва.

---

## 4. Примеры ответов разных моделей

### Пример 1. Отзыв про хорошую камеру и слабую батарею

Текст отзыва:

> «Купил DJI Mini 3 Pro. Камера отличная, но батарея садится быстро. За свои деньги норм.»

```json
// GPT-4.1 Nano
{
  "sentiment": "нейтральный",
  "main_topic": "качество камеры",
  "issue": "батарея садится быстро",
  "rating": 3
}
// Grok 4.1 Fast
{
  "sentiment": "нейтральный",
  "main_topic": "Отличная камера, но быстрая разрядка батареи",
  "issue": "Батарея садится быстро.",
  "rating": 3
}
// DeepSeek Chat
{
  "sentiment": "нейтральный",
  "main_topic": "камера и батарея",
  "issue": "батарея садится быстро",
  "rating": 3
}




Во всех трёх случаях тональность и оценка совпадают, различия — только в формулировке main_topic: Nano фокусируется на камере, Grok и DeepSeek одновременно отражают и камеру, и слабую батарею.

### Пример 1. Отзыв про хорошую камеру и слабую батарею

Текст отзыва:

> «Купил DJI Mini 3 Pro. Камера отличная, но батарея садится быстро. За свои деньги норм.»

```json
// GPT-4.1 Nano
{
  "sentiment": "нейтральный",
  "main_topic": "качество камеры",
  "issue": "батарея садится быстро",
  "rating": 3
}
// Grok 4.1 Fast
{
  "sentiment": "нейтральный",
  "main_topic": "Отличная камера, но быстрая разрядка батареи",
  "issue": "Батарея садится быстро.",
  "rating": 3
}
// DeepSeek Chat
{
  "sentiment": "нейтральный",
  "main_topic": "камера и батарея",
  "issue": "батарея садится быстро",
  "rating": 3
}
```
Все модели корректно распознают отрицательную тональность. GPT-4.1 Nano ставит 2 балла и концентрируется на проблеме двигателя, тогда как Grok 4.1 Fast и DeepSeek Chat учитывают и поломку, и работу сервиса и выставляют минимальную оценку 1.

### Пример 2. Отзыв про поломку двигателя и сервис

Текст отзыва:

> «Разочарован. Через неделю полётов сломался двигатель, сервис тянет с ремонтом.»

```json
// GPT-4.1 Nano
{
  "sentiment": "отрицательный",
  "main_topic": "качество двигателя",
  "issue": "сломался двигатель после недели использования",
  "rating": 2
}
// Grok 4.1 Fast
{
  "sentiment": "отрицательный",
  "main_topic": "поломка двигателя и проблемы с сервисом",
  "issue": "Двигатель сломался через неделю полетов, сервис тянет с ремонтом.",
  "rating": 1
}
// DeepSeek Chat
{
  "sentiment": "отрицательный",
  "main_topic": "поломка двигателя",
  "issue": "двигатель сломался через неделю, сервис медлит с ремонтом",
  "rating": 1
}
```
Dсе три модели определяют отзыв как отрицательный. GPT-4.1 Nano ставит 2 балла и фокусируется на проблеме двигателя, тогда как Grok 4.1 Fast и DeepSeek Chat учитывают и поломку, и работу сервиса и выставляют минимальную оценку 1.

### Пример 3. Отзыв про маленький запас хода батареи

Текст отзыва:

> «Запас хода по батарее маленький — максимум 18 минут. Для съёмки мероприятий этого мало.»

```json
// GPT-4.1 Nano
{
  "sentiment": "отрицательный",
  "main_topic": "время работы батареи",
  "issue": "единичное время полета недостаточно для съемки мероприятий",
  "rating": 2
}
// Grok 4.1 Fast
{
  "sentiment": "отрицательный",
  "main_topic": "маленький запас хода батареи",
  "issue": "Максимум 18 минут полёта недостаточно для съёмки мероприятий.",
  "rating": 1
}
// DeepSeek Chat
{
  "sentiment": "отрицательный",
  "main_topic": "малый запас хода батареи",
  "issue": "18 минут работы недостаточно для съёмки мероприятий",
  "rating": 2
}
```
Все три модели определяют отзыв как отрицательный и выделяют одну и ту же проблему — слишком маленький запас хода батареи. Отличия в строгости оценки: Grok 4.1 Fast ставит 1 балл как максимально негативную оценку, тогда как GPT-4.1 Nano и DeepSeek Chat выставляют 2 балла.

## 5. Выводы

1. **Какая модель лучше для задачи?**  
   Для задачи анализа отзывов о дронах наиболее удобной оказалась **Grok 4.1 Fast**: у неё самый высокий средний рейтинг (3.50), устойчивый JSON-формат и самые информативные формулировки `main_topic` и `issue`.

2. **Как выглядят остальные модели.**  
   **DeepSeek Chat** по качеству близок к Grok: он тоже хорошо выделяет ключевые преимущества и проблемы, средний рейтинг чуть ниже (3.42). **GPT-4.1 Nano** даёт более короткие и простые формулировки и немного более строгие оценки (3.25), но при этом остаётся самой лёгкой и дешёвой моделью по ресурсам.

3. **Сходство по базовой задаче.**  
   Все три модели одинаково распределяют отзывы по тональностям (5 положительных, 4 отрицательных, 3 нейтральных). В задачах уровня «положительный / нейтральный / отрицательный» существенных различий между моделями практически нет.

4. **Почему Grok 4.1 Fast удобнее.**  
   Grok 4.1 Fast лучше подходит именно для продуктовой аналитики отзывов: он не только правильно определяет тональность и рейтинг, но и даёт содержательные описания `main_topic` и `issue`, часто учитывая несколько аспектов (например, продукт + сервис).

5. **Что удивило.**  
   Удивило, что при разных провайдерах и размерах моделей (GPT-4.1 Nano, Grok 4.1 Fast, DeepSeek Chat) тональности и числовые оценки почти совпадают; основное отличие между моделями — не в цифрах, а в детализации и качестве текстовых пояснений.
